<center>
    <span style="font-family:cursive; color:crimson; font-size:35px"> Content Based Recommendation System</span>
</center>    

![image info](aa.jpg)

<font color=crimson> 
Taking example, that a user might liked or watched a movie. So once you know that the user likes this movie, what you could do is to find all the similar movies based on content like director name, cast names, generes, etc. And then recommend them back to the user.
</font>

## Read Data

In [3]:
import pandas as pd

data= pd.read_csv("movie_dataset.csv")

print("dataset shape :", data.shape)
pd.set_option("max_columns", 24)
data.head(3)

dataset shape : (4803, 24)


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


<font color=purple> 
    Since this is a content based recommendation system, we require only those columns that can specify the content of a movie.<br>
    So taking into consideration `genres` `keywords` `cast` `director` and `title` 
</font>

In [4]:
data= data[["title", "genres", "keywords", "cast", "director"]]
data.head()

,title,genres,keywords,cast,director
0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


## Checking missing values

In [5]:
data.isnull().sum()

title         0
genres       28
keywords    412
cast         43
director     30
dtype: int64

<font color=purple> 
As these columns only specify the content for the movie type, having missings will not affect the algorithm. <br>
 So replacing these missing values just by space.   
</font>

In [6]:
data= data.fillna("")

<font color=purple> 
So now we have a content, but they are in different columns, so combining them into single column.   
</font>

In [7]:
def content(row):
    return row["genres"]+ " " + row["keywords"] + " " + row["cast"] + " " + row["director"]

In [8]:
data["combined_features"] = data.apply(content, axis=1)
data.drop(columns=["genres", "keywords", "cast", "director"], axis=1, inplace=True)

In [9]:
data.head()

,title,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,Spectre,Action Adventure Crime spy based on novel secr...
3,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,John Carter,Action Adventure Science Fiction based on nove...


<font color=purple> 
Our job is to find the similarity between the movie which the user has liked or watched and the other movies.<br>
 For this first create a matrix which counts the number of words each movie content have.   
</font>

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# creating object
cv= CountVectorizer()

# creating matrix for combined_features
count_matrix= cv.fit_transform(data["combined_features"])
print(count_matrix)

  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


<font color=purple> 
Converting above count_matrix to array.

</font>

In [19]:
print(count_matrix.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


<font color=purple> 
Finding similarity score between the contents. 
</font>

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

cosineSimilarity= cosine_similarity(count_matrix)
print(cosineSimilarity)

[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]


## Explanation of above similarity scores:
<font color=purple> 
Each row and each column represents the movie.<br>
Value at (0,0) i.e `1`, means 1st movie is 100% similar to 1st movie.<br>  
Value at (0,1) i.e `0.10540926`, means 1st movie is 10.5% similar to 2nd movie.<br>   
Value at (0,2) i.e `0.12038585`, means 1st movie is 12% similar to 3rd movie.<br>
And so on.    
</font>

# Now we have the similarity scores for each movie, let's see how it is recommended to the user based on his liked or watched movie.

<font color=purple> 
Suppose the user has liked `The Dark Knight Rises`

</font>

In [21]:
user_input= "The Dark Knight Rises"

<font color=purple> 
Since, in the cosine similarity matrix, the rows and columns indicates the index of movie.<br>
So, create a function to render index.    
</font>

In [42]:
# function to get index of movie from movie name
def get_index(title):
    index=data[data["title"] == title].index.values[0]
    return index

<font color=purple> 
Index of `The Dark Knight Rises`   
</font>

In [43]:
index= get_index(user_input)
index

3

<font color=purple> 
Returning similarity scores for `The Dark Knight Rises` having index 3.  
</font>

In [72]:
similar_movies = cosineSimilarity[index]
similar_movies

array([0.03774257, 0.03580574, 0.16357216, ..., 0.03922323, 0.        ,
       0.        ])

<font color=purple> 
1st movie `Avatar` is 3.77% similar to `The Dark Knight Rises`<br>
2nd movie `Pirates of the Caribbean: At World's End` is 3.58% similar to `The Dark Knight Rises`    
                                            
</font>

### Representing the above similarity scores along with their movie index.   

In [73]:
similar_movies= list(enumerate(similar_movies))
similar_movies

[(0, 0.03774256780481986),
 (1, 0.03580574370197164),
 (2, 0.16357216402190616),
 (3, 0.9999999999999997),
 (4, 0.03922322702763681),
 (5, 0.07844645405527362),
 (6, 0.0),
 (7, 0.03580574370197164),
 (8, 0.08362420100070908),
 (9, 0.12009611535381537),
 (10, 0.11766968108291043),
 (11, 0.23533936216582085),
 (12, 0.04089304100547654),
 (13, 0.03922322702763681),
 (14, 0.14824986333222023),
 (15, 0.0),
 (16, 0.03522349768381735),
 (17, 0.04181210050035454),
 (18, 0.07161148740394328),
 (19, 0.04181210050035454),
 (20, 0.04003203845127179),
 (21, 0.04003203845127179),
 (22, 0.0),
 (23, 0.0),
 (24, 0.06933752452815364),
 (25, 0.08770580193070293),
 (26, 0.03706246583305506),
 (27, 0.08006407690254358),
 (28, 0.08006407690254358),
 (29, 0.1254363015010636),
 (30, 0.07692307692307693),
 (31, 0.07844645405527362),
 (32, 0.07412493166611012),
 (33, 0.07412493166611012),
 (34, 0.0),
 (35, 0.08559209850218259),
 (36, 0.08006407690254358),
 (37, 0.0),
 (38, 0.038461538461538464),
 (39, 0.1153846

<font color=purple> 
Sorting similarity scores in descending order with the most similar movie being at the top.
</font>

In [74]:
# function to sort the above list on the basis of it's 2nd element (similarity scores)
def secondElement(value):
    return value[1]

similar_movies.sort(key= secondElement, reverse=True)
similar_movies= similar_movies[1:]
similar_movies

<font color=purple> 
Now we have to get the title of these  movies.
</font>

In [75]:
# function to get movie name from it's index
def get_movieName(index):
    movie_name= data[data.index == index]["title"].values[0]
    return movie_name

<font color=purple> 
Printing top 10 similar movies `The Dark Knight Rises`
</font>

In [76]:
i=0
for movie in similar_movies:
    print(get_movieName(movie[0]))
    i=i+1
    if i>=10:
        break


Batman Begins
The Dark Knight
Amidst the Devil's Wings
The Killer Inside Me
The Prestige
Batman Returns
Batman
Batman & Robin
Kick-Ass
RockNRolla
